In [1]:
from etl.load_dataset import DatasetProcessor, get_tf_eggim_patch_dataset
from custom_models.cnns import base_resnet50
import tensorflow as tf
from keras.metrics import Precision, Recall, AUC, CategoricalAccuracy
from optimization.custom_losses import weighted_categorical_crossentropy
from custom_models.augmentation import basic_plus_color_augmentation, basic_augmentation
from custom_models.optimization_utilities import get_standard_callbacks
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from evaluation.metrics import categorical_accuracy, sensitivity_per_class, specificity_per_class
import pandas as pd
from sklearn.model_selection import StratifiedKFold

2025-01-22 22:13:31.601914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737584011.618238   24732 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737584011.622598   24732 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 22:13:31.638105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
target_dir = '../2025-01_EGGIM_Dataset3'  # aqui foi o o que eu chamei no remote server, mas fica a criterio
patient_ids = np.load('configs/new_patient_ids.npy', allow_pickle=True)
dp = DatasetProcessor(target_dir)
df = dp.process()
togas_ids_boolean = np.array([x.startswith('PT') for x in df['patient_id'].values])
df_togas = df[togas_ids_boolean].reset_index(drop=True)
df_ipo = df[~togas_ids_boolean].reset_index(drop=True)


In [3]:
print(len(df.index),len(df_togas.index),len(df_ipo.index))

1280 866 414


In [4]:
df.shape

(1280, 6)

In [5]:
batch_size = 32
num_epochs = 2
learning_rate = 1e-4
num_folds = 5
n_classes =3

name = f'../logs/cv_patient_resnet_multi_{num_folds}'

In [7]:
split = dp.patient_wise_split_x_fold(df_togas,df_ipo,patient_ids,n_folds=num_folds,internal_train_size=0.9,target_variable='eggim_square',random_state=42)
i = 0
for fold, (df_train, df_val, df_test) in enumerate(split):
    tf_train_df = get_tf_eggim_patch_dataset(df_train,num_classes=3,
                                             augmentation_fn=basic_augmentation,
                                             preprocess_fn=tf.keras.applications.resnet.preprocess_input)
    tf_val_df = get_tf_eggim_patch_dataset(df_val,
                                           num_classes=3,
                                           preprocess_fn=tf.keras.applications.resnet.preprocess_input)
    tf_test_df = get_tf_eggim_patch_dataset(df_test,
                                            num_classes=3,
                                            preprocess_fn=tf.keras.applications.resnet.preprocess_input)
    y_train = df_train['eggim_square']
    class_counts = np.bincount(y_train)
    class_weights_manual = {i: len(y_train) / (len(class_counts) * class_counts[i]) for i in
                                range(len(class_counts))}
    weights = tf.constant(list(class_weights_manual.values()), dtype=tf.float32)

    tf_train_df = tf_train_df.batch(batch_size)
    tf_val_df = tf_val_df.batch(batch_size)
    tf_test_df = tf_test_df.batch(batch_size)

    n_classes = 3  # Replace with the number of classes you have
    model = base_resnet50(input_shape=(224, 224, 3), n_classes=n_classes)
        # Compile the model with Adam optimizer 13:21
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss=weighted_categorical_crossentropy(weights),
                      metrics=[CategoricalAccuracy(name='cat_accuracy'), Precision(name='precision'),
                               Recall(name='recall'),
                               AUC(name='auc')])

    name_fold = name + f'fold_{fold}'
    checkpoint_dir, callbacks = get_standard_callbacks(name_fold, learning_rate)
    model.fit(tf_train_df,
                  validation_data=tf_val_df,
                  epochs=num_epochs,
                  callbacks=callbacks)
    model.load_weights(f'{checkpoint_dir}/weights.weights.h5')
    model.evaluate(tf_test_df)
       

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Epoch 1/2
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - auc: 0.6494 - cat_accuracy: 0.4820 - loss: 1.3192 - precision: 0.4877 - recall: 0.3909
Epoch 1: val_loss improved from inf to 3.26829, saving model to ../logs/cv_patient_resnet_multi_5fold_0_20250122-222454/weights.weights.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 19s 232ms/step - auc: 0.6500 - cat_accuracy: 0.4825 - loss: 1.3245 - precision: 0.4883 - recall: 0.3920 - val_auc: 0.3603 - val_cat_accuracy: 0.2254 - val_loss: 3.2683 - val_precision: 0.2254 - val_recall: 0.2254 - learning_rate: 1.0000e-04
Epoch 2/2
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - auc: 0.5427 - cat_accuracy: 0.4101 - loss: 2.0407 - precision: 0.4196 - recall: 0.3933
Epoch 2: val_loss improved from 3.26829 to 2.15696, saving model to ../logs/cv_patient_resnet_multi_5fold_0_20250122-222454/weights.weights.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - auc: 0.5494 - cat_accuracy: 0.4157 - loss: 2.0115 - precision: 0.4257 - recall: 0.3984 - val_auc: 0.3983 - val_cat_accuracy: 0.23

In [8]:
model_weights_directory = '../logs/'
model_fold_dir = sorted(os.listdir(model_weights_directory))

split = dp.patient_wise_split_x_fold(df_togas,df_ipo,patient_ids,n_folds=num_folds,internal_train_size=0.9,target_variable='eggim_square',random_state=42)
eggim_values = []

for fold, (df_train, df_val, df_test) in enumerate(split):
    patients = df_test['patient_id'].unique()

    n_classes = 3  # Replace with the number of classes you have
    model = base_resnet50(input_shape=(224, 224, 3), n_classes=n_classes)
    # Compile the model with Adam optimizer 13:21
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=[CategoricalAccuracy(name='cat_accuracy'), Precision(name='precision'),
                           Recall(name='recall'),
                           AUC(name='auc')])


    model.load_weights('../logs/' + model_fold_dir[fold] + '/weights.weights.h5')
    
    for patient in patients:
        df_patient = df_togas[df_togas['patient_id'] == patient]
        df_patient = df_patient[~df_patient['landmark'].duplicated(keep='first')]
        landmarks = df_patient['landmark'].to_list()
        tf_df_patient = get_tf_eggim_patch_dataset(df_patient,
                                            num_classes=3,
                                            preprocess_fn=tf.keras.applications.resnet.preprocess_input) 
        tf_df_patient = tf_df_patient.batch(batch_size)

        y_true = np.concatenate([y for (_, y) in tf_df_patient])
        y_pred = model.predict(tf_df_patient)
        y_true_ordinal = np.argmax(y_true, axis=-1) # [0 0 1] -> 2
        y_pred_ordinal = np.argmax(y_pred, axis=-1)

        eggim_true = 0
        eggim_pred = 0

        first_group = 0
        first_group_value_true = 0
        first_group_value_pred = 0

        last_group = 0
        last_group_value_true = 0
        last_group_value_pred = 0

        for i in range(len(landmarks)):
            if landmarks[i][:2] == 'vi' :
                eggim_true += y_true_ordinal[i]
                eggim_pred += y_pred_ordinal[i]
            elif landmarks[i][:2] == 'ix' or landmarks[i][:2] == 'x.':
                if first_group == 0:
                    first_group = 1
                    first_group_value_true = y_true_ordinal[i]
                    first_group_value_pred = y_pred_ordinal[i]
                else:
                    first_group_value_true = (first_group_value_true + y_true_ordinal[i]) /2
                    first_group_value_pred = (first_group_value_pred + y_pred_ordinal[i]) /2
            else:
                if last_group == 0:
                    last_group = 1
                    last_group_value_true = y_true_ordinal[i]
                    last_group_value_pred = y_pred_ordinal[i]
                else:
                    last_group_value_true = (last_group_value_true + y_true_ordinal[i]) /2 
                    last_group_value_pred = (last_group_value_pred + y_pred_ordinal[i]) /2

        eggim_true += first_group_value_true
        eggim_pred += first_group_value_pred
        eggim_true += last_group_value_true
        eggim_pred += last_group_value_pred

        eggim_values.append([patient,eggim_true,eggim_pred]) 
    
;

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-01-22 22:41:18.272266: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


2025-01-22 22:41:21.040931: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


2025-01-22 22:41:21.399941: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-01-22 22:41:23.499335: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1681_0', 8 bytes spill stores, 8 bytes spill loads

2025-01-22 22:41:24.278392: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1688', 4 bytes spill stores, 4 bytes spill loads

2025-01-22 22:41:24.293579: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1688', 44 bytes spill stores, 44 bytes spill loads



...

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


2025-01-22 22:42:04.837064: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


''

In [9]:
eggim_values

[['PT004', 1.0, 9.0],
 ['PT005', 3.0, 9.0],
 ['PT006', 2.0, 9.0],
 ['PT011', 5.0, 10.0],
 ['PT013', 1.0, 6.5],
 ['PT015', 0.0, 10.0],
 ['PT016', 0.0, 10.0],
 ['PT020', 2.0, 9.5],
 ['PT023', 1.5, 8.0],
 ['PT024', 7.0, 10.0],
 ['PT030', 0.5, 8.0],
 ['PT032', 4.0, 9.5],
 ['PT037', 1.0, 10.0],
 ['PT021', 8.0, 10.0],
 ['PT025', 3.0, 9.0],
 ['PT026', 2.0, 9.0],
 ['PT028', 5.0, 8.5],
 ['PT029', 7.0, 10.0],
 ['PT031', 0.0, 10.0],
 ['PT036', 2.5, 9.0],
 ['PT041', 1.0, 8.5],
 ['PT046', 1.0, 9.5],
 ['PT047', 0.0, 10.0],
 ['PT048', 1.0, 8.0],
 ['PT049', 1.0, 10.0],
 ['PT059', 4.5, 9.0],
 ['PT009', 9.0, 10.0],
 ['PT022', 8.0, 10.0],
 ['PT034', 3.5, 10.0],
 ['PT039', 2.5, 9.0],
 ['PT040', 6.0, 10.0],
 ['PT043', 5.5, 10.0],
 ['PT051', 0.5, 10.0],
 ['PT061', 1.0, 10.0],
 ['PT062', 4.0, 10.0],
 ['PT063', 1.0, 10.0],
 ['PT064', 1.5, 10.0],
 ['PT065', 0.0, 9.0],
 ['PT066', 1.0, 8.5],
 ['PT035', 8.5, 10.0],
 ['PT038', 3, 10],
 ['PT045', 5.0, 10.0],
 ['PT050', 3.0, 9.5],
 ['PT053', 6.0, 10.0],
 ['PT054', 2

In [10]:
y_true = []
y_pred = []
for values in eggim_values:
    if values[1] < 5: y_true.append(0)
    else: values[1] = y_true.append(1)
    if values[2] < 5: y_pred.append(0)
    else: values[2] = y_pred.append(1)


In [11]:
cm = confusion_matrix(y_true,y_pred)
cm_df = pd.DataFrame(cm, index=[f"True {name}" for name in ['0','1']],
                     columns=[f"Pred {name}" for name in ['0','1']])

cm_df

,Pred 0,Pred 1
True 0,0,49
True 1,0,16


In [4]:
def replace_landmark_name(landmark):
    new_landmark_name = {'ii': 'ii',
                    'ix': 'ix',
                    'vi': 'vi',
                    'vii': 'vii',
                    'viii': 'viii',
                    'x': 'x',
                    'xii': 'xii'}
    landmark_number = landmark.split('.')[0]
    return new_landmark_name[landmark_number]

In [5]:
df_patients = df_togas[df_togas['patient_id'].isin(patient_ids)].drop(['bbox','eggim_global','image_directory'],axis=1)
df_patients = df_patients.drop_duplicates(subset=['patient_id','landmark'])
df_patients['landmark'] = df_patients['landmark'].apply(replace_landmark_name)

df_patients[df_patients['patient_id'] == 'PT081']

,eggim_square,landmark,patient_id
807,1,vii,PT081
808,1,vi,PT081
809,1,viii,PT081
810,2,ix,PT081
811,0,xii,PT081


In [6]:
filtered_df = df_patients[df_patients['landmark'].isin(['ix','x'])]
filtered_df = filtered_df.groupby(['patient_id'], as_index=False).agg({'eggim_square' : 'mean', 'landmark' : lambda x : 'ix'})

df_patients = df_patients[~df_patients['landmark'].isin(['ix','x'])]

df_patients = pd.concat([df_patients, filtered_df])

In [7]:
filtered_df = df_patients[df_patients['landmark'].isin(['ii','xii'])]
filtered_df = filtered_df.groupby(['patient_id'], as_index=False).agg({'eggim_square' : 'mean', 'landmark' : lambda x : 'ii'})

df_patients = df_patients[~df_patients['landmark'].isin(['ii','xii'])]

df_patients = pd.concat([df_patients, filtered_df])

In [8]:
df_patients

,eggim_square,landmark,patient_id
8,0.0,vi,PT004
9,1.0,viii,PT004
16,0.0,vii,PT004
18,0.0,vii,PT005
19,1.0,vi,PT005
...,...,...,...
60,0.0,ii,PT081
61,0.0,ii,PT082
62,0.0,ii,PT083
63,0.0,ii,PT084


In [9]:
df_per_patient = df_patients.groupby(['patient_id'], as_index=False).sum()
df_per_patient['eggim_square'] = df_per_patient['eggim_square'].astype(int)


In [10]:
df_per_patient

,patient_id,eggim_square,landmark
0,PT004,1,viviiiviiixii
1,PT005,3,viiviviiiixii
2,PT006,2,viiiviiviixii
3,PT009,9,viiviviiiixii
4,PT011,5,viviiviiiixii
...,...,...,...
60,PT081,5,viiviviiiixii
61,PT082,5,viiviviiiixii
62,PT083,0,viiviviiiixii
63,PT084,6,viiviviiiixii


In [18]:
def get_folds(eggim,patients):
    skf = StratifiedKFold(n_splits=5,shuffle=False)
    folds = []
    for _, (_, test_index) in enumerate(skf.split(patients,eggim)):
        p_list = [patients[index] for index in test_index ]
        folds.append(p_list)

    return folds

folds = get_folds(df_per_patient['eggim_square'].to_list(),df_per_patient['patient_id'].to_list())
folds

/home/goda/Desktop/tese/venv_3.10.12/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[['PT004',
  'PT005',
  'PT006',
  'PT011',
  'PT013',
  'PT015',
  'PT016',
  'PT020',
  'PT023',
  'PT024',
  'PT030',
  'PT032',
  'PT037'],
 ['PT021',
  'PT025',
  'PT026',
  'PT028',
  'PT029',
  'PT031',
  'PT036',
  'PT041',
  'PT046',
  'PT047',
  'PT048',
  'PT049',
  'PT059'],
 ['PT009',
  'PT022',
  'PT034',
  'PT039',
  'PT040',
  'PT043',
  'PT051',
  'PT061',
  'PT062',
  'PT063',
  'PT064',
  'PT065',
  'PT066'],
 ['PT035',
  'PT038',
  'PT045',
  'PT050',
  'PT053',
  'PT054',
  'PT067',
  'PT068',
  'PT070',
  'PT071',
  'PT072',
  'PT076',
  'PT081'],
 ['PT060',
  'PT069',
  'PT073',
  'PT074',
  'PT075',
  'PT077',
  'PT078',
  'PT079',
  'PT080',
  'PT082',
  'PT083',
  'PT084',
  'PT086']]